# Publications markdown generator for academicpages

Takes a TSV of publications with metadata and converts them for use with [academicpages.github.io](academicpages.github.io). This is an interactive Jupyter notebook ([see more info here](http://jupyter-notebook-beginner-guide.readthedocs.io/en/latest/what_is_jupyter.html)). The core python code is also in `publications.py`. Run either from the `markdown_generator` folder after replacing `publications.tsv` with one containing your data.

TODO: Make this work with BibTex and other databases of citations, rather than Stuart's non-standard TSV format and citation style.


## Data format

The TSV needs to have the following columns: pub_date, title, venue, excerpt, citation, site_url, and paper_url, with a header at the top. 

- `excerpt` and `paper_url` can be blank, but the others must have values. 
- `pub_date` must be formatted as YYYY-MM-DD.
- `url_slug` will be the descriptive part of the .md file and the permalink URL for the page about the paper. The .md file will be `YYYY-MM-DD-[url_slug].md` and the permalink will be `https://[yourdomain]/publications/YYYY-MM-DD-[url_slug]`

This is how the raw file looks (it doesn't look pretty, use a spreadsheet or other program to edit and create).

In [51]:
!cat publications.tsv

pub_date	title	venue	excerpt	citation	url_slug	paper_url
01/12/2014	Simultaneous enhancement of excitation and postburst inhibition at the end of focal seizures: Focal Seizures	Annals of Neurology	It is not only the inhibition necessary to stop epileptic seizures	"<b>Boido D</b>, Gnatkovsky V, Uva L, Stefano Francione S, de Curtis M. (2014). ""Simultaneous enhancement of excitation and postburst inhibition at the end of focal seizures: Focal Seizures"". <b>Annals of Neurology</b> 76 (6)"	Boido-Seizure-Termination	https://onlinelibrary.wiley.com/doi/abs/10.1002/ana.24193
03/07/2019	Mesoscopic and microscopic imaging of sensory responses in the same animal	Nature Communications	Two-Photon Microscopy, functional US and BOLD fMRI in the same mouse	"<b>Boido D*</b>, Rungta R*, Osmanski B*, Roche M, Tsurugizawa T, Le Bihan D, Ciobanu L*, Charpak S*. (2019). ""Mesoscopic and microscopic imaging of sensory responses in the same animal"" <b>Nature Communications</b>. 10(1)."	Boido-Mesoscopic	ht

## Import pandas

We are using the very handy pandas library for dataframes.

In [52]:
import pandas as pd

## Import TSV

Pandas makes this easy with the read_csv function. We are using a TSV, so we specify the separator as a tab, or `\t`.

I found it important to put this data in a tab-separated values format, because there are a lot of commas in this kind of data and comma-separated values can get messed up. However, you can modify the import statement, as pandas also has read_excel(), read_json(), and others.

In [53]:
publications = pd.read_csv("publications.tsv", sep="\t", header=0)
publications


,pub_date,title,venue,excerpt,citation,url_slug,paper_url
0,01/12/2014,Simultaneous enhancement of excitation and pos...,Annals of Neurology,It is not only the inhibition necessary to sto...,"<b>Boido D</b>, Gnatkovsky V, Uva L, Stefano F...",Boido-Seizure-Termination,https://onlinelibrary.wiley.com/doi/abs/10.100...
1,03/07/2019,Mesoscopic and microscopic imaging of sensory ...,Nature Communications,"Two-Photon Microscopy, functional US and BOLD ...","<b>Boido D*</b>, Rungta R*, Osmanski B*, Roche...",Boido-Mesoscopic,https://www.nature.com/articles/s41467-019-090...
2,11/06/2020,Transfer functions linking neural calcium to s...,Nature Communications,"The Hemodynamic Response Function, so the neur...","Aydin A-K, Haselden WD, Goulam Houssen Y, Pouz...",Aydin-TF,https://www.nature.com/articles/s41467-020-167...
3,14/06/2021,"Iliski, a software for robust calculation of t...",PLOS Computational Biology,Introduction to a SW to compute Transfer Funct...,"Aydin AK, Haselden WD, Dang J, Drew PJ, Charpa...",Aydyn-Iliski,https://www.ncbi.nlm.nih.gov/pmc/articles/PMC8...
4,23/09/2022,Cranial window for longitudinal and multimodal...,Neurophotonics,This paper described a multimodal mouse prepar...,"Tournissac M*, <b>Boido D*</b>, Omnes M, Goula...",Tournissac-Craniotomy,https://pubmed.ncbi.nlm.nih.gov/36159711/


## Escape special characters

YAML is very picky about how it takes a valid string, so we are replacing single and double quotes (and ampersands) with their HTML encoded equivilents. This makes them look not so readable in raw format, but they are parsed and rendered nicely.

In [54]:
html_escape_table = {
    "&": "&amp;",
    '"': "&quot;",
    "'": "&apos;"
    }

def html_escape(text):
    """Produce entities within text."""
    return "".join(html_escape_table.get(c,c) for c in text)

## Creating the markdown files

This is where the heavy lifting is done. This loops through all the rows in the TSV dataframe, then starts to concatentate a big string (```md```) that contains the markdown for each type. It does the YAML metadata first, then does the description for the individual page.

In [55]:
import os
for row, item in publications.iterrows():
    
    md_filename = str(item.pub_date) + "-" + item.url_slug + ".md"
    html_filename = str(item.pub_date) + "-" + item.url_slug
    year = item.pub_date[:4]
    
    ## YAML variables
    
    md = "---\ntitle: \""   + item.title + '"\n'
    
    md += """collection: publications"""
    
    md += """\npermalink: /publication/""" + html_filename
    
    if len(str(item.excerpt)) > 5:
        md += "\nexcerpt: '" + html_escape(item.excerpt) + "'"
    
    md += "\ndate: " + str(item.pub_date) 
    
    md += "\nvenue: '" + html_escape(item.venue) + "'"
    
    if len(str(item.paper_url)) > 5:
        md += "\npaperurl: '" + item.paper_url + "'"
    
    md += "\ncitation: '" + html_escape(item.citation) + "'"
    
    md += "\n---"
    
    ## Markdown description for individual page
        
    if len(str(item.excerpt)) > 5:
        md += "\n" + html_escape(item.excerpt) + "\n"
    
    if len(str(item.paper_url)) > 5:
        md += "\n[Download paper here](" + item.paper_url + ")\n" 
        
    md += "\nRecommended citation: " + item.citation
    
    md_filename = os.path.basename(md_filename)
       
    with open("../_publications/" + md_filename, 'w') as f:
        f.write(md)

These files are in the publications directory, one directory below where we're working from.

In [56]:
!ls ../_publications/

2014-Boido-Seizure-Termination.md 2021-Aydyn-Iliski.md
2019-Boido-Mesoscopic.md          2022-Tournissac-Craniotomy.md
2020-Aydin-TF.md


In [57]:
!cat ../_publications/2022-paper-title-number-2.md

cat: ../_publications/2022-paper-title-number-2.md: No such file or directory
